In [ ]:
!unzip models/single_agent_llama/results.zip -d models/single_agent_llama/
!unzip models/orchestrator_agent_llama/results.zip -d models/orchestrator_agent_llama/
!unzip models/multi_agent_llama/results.zip -d models/multi_agent_llama/

In [ ]:
import os
import json

os.environ["TEAM_API_KEY"] = ""
os.environ["SERPAPI_API_KEY"] = ""

from aixplain.factories import ModelFactory

PROMPT = (
    "As a math expert, you will be provided with three items: a #Question#, an #Answer#, and the #Ground Truth#. " +
    "Your task is to determine whether the #Answer# matches the #Ground Truth#. " +
    "You need to considering mathmetical theorems when verifing the answer. " +
    "For example, '$(c + 9)(c - 4)$' and '(c - 4)*(c + 9)' should be the same expressions based on factorization theorems. " +
    "If they align, respond with 'correct'. If they do not, respond with 'wrong'. " +
    "Ensure that your #Response# is the final line and consists solely of the word 'correct' or 'wrong', without any additional commentary or explanation. " +
    "\n\n#Question#:\n{question}\n\n#Answer#:\n{answer}\n\n#Ground Truth#:\n{ground_truth}\n\n#Response#: "
)

evaluator = ModelFactory.get("66b2708c6eb5635d1c71f611")

with open("data/math_standard_20240712.jsonl") as f:
    data = [json.loads(line) for line in f.readlines()]

In [ ]:
for model in ["single_agent_llama", "orchestrator_agent_llama", "multi_agent_llama", "multi_agent_llama_reflexion"]:
# for model in ["single_agent_llama"]:
    y_pred ={}
    for idx, row in enumerate(data):
        real = row["gt_answer"]
        for i in range(1):
            if str(i) not in y_pred:
                y_pred[str(i)] = []
            with open(f"models/{model}/results/{idx}_{i}.json") as f:
                pred = json.load(f)["output"].split("#ANSWER#:")[-1]
                
            response = evaluator.run(PROMPT.format(question=row["raw_problem"], answer=pred, ground_truth=real))
            if response.status == "SUCCESS":
                y_pred[str(i)].append(response.data.strip())
            else:
                y_pred[str(i)].append("error")
    
    with open(f"models/{model}/results.json", "w") as f:
        json.dump(y_pred, f)


In [12]:
for model in ["single_agent_llama"]:
    with open(f"models/{model}/results.json") as f:
        y_pred = json.load(f)
    for i in range(1):
        accuracy = sum([1 if x == "correct" else 0 for x in y_pred[str(i)]]) / len(y_pred[str(i)])
        print(f"Model {model} - Run {i} - Accuracy: {accuracy}")
    print(10 * "-")

Model single_agent_llama - Run 0 - Accuracy: 0.634
----------
